# day02

## VCC와 GND

### VCC

- VCC와 VDD(TTL/CMOS에 따라서 달라짐)는 같음

### GND

- Ground
- 기준 전압, 0v
- Vss라고도 함

## 쇼트란?

- Short == short-circuit == 단락
- +와 -가 바로 연결되거나, 저항값이 0에 가까운 경우

## AC와 DC

- 직류: 항상 같은 방향으로 전류 이동
- 교류: 시간의 경과에 따라 흐르는 방향이 바뀜
  - 이유: 발전기의 회전 운동의 영향
  - 발전기의 회전수: 60hz
  - 장점: 멀리 송신 가능

## 풀업 저항

- 입력 핀에는 항상 HIGH 혹은
- 전위를 하위 레벨로 확실히 끌어올려주기 때문에 풀업 저항
- 전류 낭비를 막기 위해서 풀업으로 함

## Hi-Z

- High Impedance == 저항
- 마치 선이 끊어진 것 같은 상태(절연 상태)
- High나 Low가 아닌 제3의 상태
- 출력저항이 큰 상태
  - 저항이 많다는 것은 부도체가 됐다는 것
  - 부도체가 됐다는 것은 전기가 안 통한다는 것
  - 출력 상태를 제어하지 않음
- SS로 구분한다.

## 삼상(3-state)

- 0, 1, HI-Z라는 3가지 상태를 표현
  - 이 3가지 표현
- 포트를 사용하지 않을 때 HIGH-Z 상태로 둔다.
- 입력에 반응하지 않는다.
- 언제 접하게 되나?
  - JTAG 핀(PORTF)

## Active Low

- 0(Logic Low)인 상태를 "참"으로 보는 상태

## 문의

- 리셋은 원래 0으로 되어 있고 우리가 스위치를 눌렀을 때 1이 되는거 아닌가요?

## Rising Edge와 Falling Edge

- 전기 신호의 Logic이 변하는 시점
- Rising Edge: 0v -> 5v로 올라가는 시점
- Falling Edge: 5v -> 0v로 떨어지는 시점
- 데이터 동기화 작업에 사용 됨

## Datasheet?

- 전자 부품의 스펙, 특성, 사용법이 정리된 문서
- 즉, 특정 부품에 대한 사용 매뉴얼
- 모델명 datasheet 라고 검색하면 됨 

## 전기 관련 단위 체계

Abbreviation | Means | Multiply unit by | Or
--- | --- | --- | ---
p | pico | .000000000001 | 10^-12

## 온도 센서 사용 실습

- Analog to Digital Convert의 약자

```c
#define F_CPU 8000000UL 
#include <avr/io.h> 
#include <util/delay.h>

void Puts(char *data) {
	int i;
	for(i=0; i< strlen(data); i++) {
		while((UCSR0A & 0x20) == 0);
        UDR0 = data[i]; 
	}
}

int main(void) {
	char result[20]; 
	int value;
	DDRA = 0xff; 
	PORTA = 0xff;
	UCSR0A = 0x00; 
	UCSR0B = 0x98; 
	UCSR0C = 0x06; 
	UBRR0H = 0; 
	UBRR0L = 47;
	
	while(1)
	// bps == 9600
	{
		ADMUX=0; //ADC설정초기화시작 ADCH = 0;
		ADCL = 0;
		ADCSRA = 0xc7;
		
		// 설정이 완료될 때까지 기다림 
		while((ADCSRA & 0x10) == 0);
		
		// ADC0의 상하위 바이트 값을 이용하여 데이터 복원 
		value = ADCL | (ADCH << 8);
		
		memset(result, 0, 20);
		sprintf(result, "Temporature : %d\r\n", (value*100/205)); 
		Puts(result);
		_delay_ms(1000);
	} 
}
```

- ADMUX: 기본 세팅
- ADCSRA: ADC 변환을 시작하는 단계
- 분주비: 분해 능력
- (value * 100/205)
  - 100도 일때 205로 매치가 됨
  - 100: 205 = x : value
  - 205x = value * 100
  - x = value * 100/205

```c
#include <avr/io.h> 
#include <avr/interrupt.h>

ISR(INT0_vect)
{
    PORTA = 0xff;
}

int main(void)
{
    DDRA = 0xff;
    DDRC = 0xff;
    PORTC = 0xff;
    
    EIMSK = 0b00000001; /* INT0 ENABLE */
    EICRA = 0b00000011; /* INT0 -> rising edge로 설정 */
    EIFR = = 0b00000000; /* flag clear */
    sei(); // 인터럽트 기능 활성화
    
    while(1) { // 다른 일 ⋯ 
    }
}
```

- EIMSK: MCU의 INT0 인터럽트 기능을 활성화 시킴

## 타이머로 숫자 세기

- 타이머란?
- TIMSK(Timer Interrupt Mask Register)
- TCCR0(

## 분주기(Prescaler)

- 여러 클럭을 묶어서 가상의 클럭을 만드는 것
- 결과적으로 클럭이 느려지는 효과를 얻을 수 있음

## PWM(Pulse Width Modulation)

- LED의 밝기를 반으로 줄일 수는 없을까?
  - PWM을 이용하면 가능

## 무선통신 실습

```c
#define F_CPU 16000000UL 
#include <avr/io.h> 
#include <string.h> 
#include <util/delay.h> 
#include <avr/interrupt.h>

void Puts(char *data){
    int i;
    for(i=0; i<strlen(data); i++) { 
        while((UCSR0A & 0x20) == 0); 
        UDR0 = data[i];
    } 
}

ISR(INT0_vect){
    Puts("IR signal received\r\n");
}

int main(void) {
    DDRA = 0xff; 
    PORTA = 0xff; 
    DDRB = 0xff; 
    PORTB = 0xff;
    
    UCSR0A = 0x00; UCSR0B = 0x98; UCSR0C = 0x06; UBRR0H = 0;UBRR0L = 103; // bps == 9600
    
    EIMSK = 0b00000001; /* INT0 ENABLE */
    EICRA = 0b00000010; /* INT0 -> falling edge로 설정 */
    EIFR = 0b00000000; /* flag clear */
    sei();
    
    while(1) {} 
}
    
    
    
```

- 첫번째 비트는 무조건 떨어지게 만들고 나머지 8비트를 의미있는 것으로 본다
- 

# 2일차 하드웨어 통신

## 하드웨어 통신이 필요한 경우

- 주변 장치와의 통신
  - CAMERA 모듈
  - Zigbee 무선 통신 모듈
- MCU와 MCU 통신
- 무엇으로 "Interfacing" 할 것인가?

## 동기와 비동기

- 동기
  - Clock을 기준으로 데이터 참조
  - 최소 2회선 필요
  - 시작/종료 시그널 불필요
- 비동기
  - 시간을 기준으로 데이터 참조
  - 최소 1회선 필요
  - 시작/종료 시그널 필요

## 1. UART

- Universial asynchronous receiver/transmitter
  - 범용 비동기 송/수신기
- 가장 오래 된 하드웨어 통신 프로토콜 중 하나
- 직렬 통신 프로토콜
  - 데이터 송신/수신 시 각각 하나의 LINE만 이용
- 2개의 라인 사용
- Baud rate로 ㄴㅏ눔
- 디버깅

## 2. I2C

- Inter-Integrated Circuit

## 3. SPI

- Serial Peripheral Interface
- 1:N 통신 가능
- 3개의 라인 사용
  - MISO, MOSI, SCK
- MISO: Master-In, Slave-Out
- MOSI: Master-Out, Slave-In
- Socket의 server/client와 비슷한 개념

- [WormFood's AVR Baud Rate Calculator Ver. 1.3.2](http://wormfood.net/avrbaudcalc.php)
- [Logic Analyzer Downloads](https://www.saleae.com/downloads)


```c
#define F_CPU 16000000UL 
#include <avr/io.h> 
#include <util/delay.h>

int main(void)
{
	/* Status Register 0A */ 
	UCSR0A = 0x00;
	
	/* Status Register 0B */
	/* RX/TX Enable = 10011000 */ 
	UCSR0B = 0x98;
	
	/* Status Register 0C */
	/* No parity, 8bit = 0110 */ 
	UCSR0C = 0x06;
	
	/* 중요 : Baud Rate 설정 */ /* BPS = 9600 */
	UBRR0H = 0;
	UBRR0L = 103;
	
	while(1) {
		UDR0 = 'A';
		_delay_ms(1000); 
	}
}
```

## 데이터 비트 분석

- LSB가 먼저 지나가게 약속이 되어 있음
- 가장 낮은 비트가 먼저 나오게 되어 있음
- LSB, MSB

## 해커가 UART를 통해 얻을 수 있는 것들

- 커널, OS 메시지
  - 취약점 공략에 필요한 각종 정보 획득
- 디버그 메시지
  - ex) printf("initializing network adaptor ok\n");
- 오류 메시지
  - ex) Segmentation fault, command not found
- Hidden or Setting Menu
- 부트로더(Bootloader)
  - 펌웨어 획득
  - 새로운 펌웨어 Writing
- 커맨드 쉘(Command Shell)
  - 펌웨어, 바이너리 획득
  - 동적 분석 가능

## 쇼단 검색

- 웹 포트가 열려져 있어야 하고
- UART는 뜯어서 눈으로 봐야 한다.

## jmod 에서 vcc 하는 방법?

## UART 연결 절차 요약

- CP2102, PL2303, 
- 1: VCC
- 2: TX
- 3: RX
- 4: GND
- 보레이트: 38400

- 통신이 안될 때 어떻게 하는가? 

## 스마트폰 JIG

- JIG? 다른 무언가를 연결하기 위한 틀
- USB 핀의 저항값을 체크하여 구분

### adb shell

- adb shell이 활성화가 안 되어 있으면 안됨
- 핸드폰 키면 패턴 걸려져 있는 폰 해킹 못하는데 패턴이 걸려져 있어도 할 수 있음
- 5초 안에 백도어 설치
- 제스쳐.db 크랙. 제스쳐.db 지우면 된다고 함

## 후기

- 1일차에 모두 진행하지 못했던 부분부터 진행
- 정전압 어댑터의 용량에서 알려주신 것처럼 2A는 엄청 큰 전류!!
- 전기의 위험성
- 전압 차이의 중요성: 서로 전위 차이가 발생하면 전류가 흐르게 된다. 전위 차이만 발생하지 않으면 고압 전선을 만져도 상관이 없지만 직접 실험은 비추
- 풀업(Pull-Up) 저항: 입력 핀에는 항상 HIGH 혹은 LOW 신호를 주어야 함
  - 신호가 없다면 HIGH도 LOW도 아닌 Floating 상태가 됨
  - Pull-Up의 의미: 전위를 High 레벨로 확실하게 끌어 올려줌
- 풀다운(Pull-Down) 저항: 전위를 Low 레벨로 확실하게 끌어 내려줌
- 풀업/풀다운의 중요성
  - HIGH와 LOW를 명확히 하기 위하여(Floating 상태 방지)
- 전류 소모 방지
  - 풀업 상태일 때 불필요한 전류 흐름 차단
  - 기본 풀다운 상태로 놔두면 전류가 계속 GND로 흐름
- 오작동 방지
  - 만약 어떤 핀이 외부 메모리나 센서에 연결되어 있다고 가정할 때, 풀업/풀다운 처리가 되어있지 않다면 초기화 작업 전까지 어떤 짓을 할지 모른다.

## 키보드 key값

In [6]:
a = 0b00011100
a

28

In [7]:
hex(a)

'0x1c'

In [10]:
chr(0x41)

'A'

## 키보드

- CLOCK/DATA 확대
- 첫번째 A, 세번째 A, 두번째가 뗀다는 신호
- 0, 10번째는 start, 종료 비트
- 낮은 비트가 먼저 가지 않았을까? 그래서 우리가 볼 때는 뒤집어서 봐야한다.
- ascii 코드 A, 소프트웨어에 디파인되어 있는 것. 하드웨어라 아스키코드의 개념이 없는 공간이다.
- 키맵 테이블을 검색해보면 됨
- 데이터: PORTA
- 클럭: 인터럽트로 처리할 것이기 때문에 PD0
- 인터럽트를 falling edge에 건다. 인터럽트가 뜰 때마다 1비트씩 가져와서 화면에 출력까지
- timer를 써서 1초에 몇 번씩 반복해서 뿌려준다?
- 새로운 key를 받을 수 없을 때는 clock을 내려버린다.
- 비트를 모아서 처리를 할 시간이 필요하다. 처리를 해야되기 때문에 시간이 필요하다. 그래서 clock을 0으로 내려버림
- 키보드의 임시 버퍼에 잠깐 머물러 있다가 보내주게 된다. 1024바이트가 있다.


- 전원이 들어 와 있는 상태라면 vcc 불필요
- [Interfacing the PC's Keyboard.](http://retired.beyondlogic.org/keyboard/keybrd.htm)


## AVR을 이용하여 키보드 신호 스니퍼 실습 문제

- 힌트: 인터럽트 사용
- 목표1: DATA 신호를 0과 1로 UART 출력
- 목표2: DATA 신호를 분석하여 문자로 출력

```c
#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>
#include <avr/interrupt.h>

void Puts(char *data)
{
	int i;
	for(i=0; i<strlen(data); i++)
	{
		while((UCSR0A & 0x20) == 0);
		UDR0 = data[i];
	}
}

ISR(INT0_vect)
{
	if(PINA & 0x01)
	Puts("1");
	else
	Puts("0");
}


int main(void)
{
	
	DDRA = 0x00;

	UCSR0A = 0x00;
	UCSR0B = 0x18;
	UCSR0C = 0x06;
	UBRR0H = 0;
	UBRR0L = 8;

	Puts("KEYBOARD SNIFFING START!\r\n");


	EIMSK = 0b00000001;    /* INT0 ENABLE */
	EICRA = 0b00000010;    /* INT0 -> falling edge로 설정 */
	EIFR = 0b00000000;     /* flag clear */
	sei();


	while(1)
	{
	}
}
```

In [12]:
string = '''000111000011000001111111000111000011'''

In [17]:
len(string)

36

In [27]:
0x41 << 3

520

### 키 입력 받아서 출력

```c


#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>

void Puts(char *data)
{
	int i;
	for(i=0; i<strlen(data); i++)
	{
		while((UCSR0A & 0x20) == 0);
		UDR0 = data[i];
	}
}


char key_value = 0;
int count = 0;

ISR(INT0_vect)
{
	int bit;

	if(count > 0 && count < 9)
	{

		bit = PINA & 0x01;

		key_value = key_value | bit << (count - 1);


	/*
			받은 값
			0 0 1 1 1 0 0 0

			count 1 : 00000000
			count 2 : 00000000
			count 3 : 00000100
			count 4 : 00001100
			count 5 : 00011100
			count 6 : 00011100
			count 7 : 00011100
			count 8 : 00011100
		
			최종 값
			0 0 0 1 1 1 0 0
	*/

		
	}

	count++;

	if(count == 9)
	{
		sprintf(message, "key %x is pressed\r\n", key_value);
		Puts(message);

		/*
		if(key_value == 0x1c)
			Puts("Key 'A' is pressed\r\n");

		*/

		count = 0;
		key_value = 0;
	}
}


int main(void)
{
	
	DDRA = 0x00;

	UCSR0A = 0x00;
	UCSR0B = 0x18;
	UCSR0C = 0x06;
	UBRR0H = 0;
	UBRR0L = 8;

	Puts("KEYBOARD SNIFFING START!\r\n");

    EIMSK = 0b00000001;    /* INT0 ENABLE */
    EICRA = 0b00000010;    /* INT0 -> falling edge濡� �ㅼ젙 */
    EIFR = 0b00000000;     /* flag clear */
    sei();

	while(1)
	{
	}
}

```